In [8]:
import copy

# Note: the initialization code for the sudoku board along with the beautiful ASCII printing was taken from:
# https://stackoverflow.com/questions/45471152/how-to-create-a-sudoku-puzzle-in-python
base  = 3

side  = base*base

def create_board():
    # pattern for a baseline valid solution
    def pattern(r,c): return (base*(r%base)+r//base+c)%side

    # randomize rows, columns and numbers (of valid base pattern)
    from random import sample
    def shuffle(s): return sample(s,len(s)) 
    rBase = range(base) 
    rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
    cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
    nums  = shuffle(range(1,base*base+1))
    
    board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
    answer = copy.deepcopy(board)
    squares = side*side
    empties = squares * 3//4
    for p in sample(range(squares),empties):
        board[p//side][p%side] = 0
    return answer, board

In [9]:
def print_board(board):
    base = 3
    def expandLine(line):
        return line[0]+line[5:9].join([line[1:5]*(base-1)]*base)+line[9:13]

    line0  = expandLine("╔═══╤═══╦═══╗")
    line1  = expandLine("║ . │ . ║ . ║")
    line2  = expandLine("╟───┼───╫───╢")
    line3  = expandLine("╠═══╪═══╬═══╣")
    line4  = expandLine("╚═══╧═══╩═══╝")

    symbol = " 1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    nums   = [ [""]+[symbol[n] for n in row] for row in board ]
    print(line0)
    for r in range(1,side+1):
        print( "".join(n+s for n,s in zip(nums[r-1],line1.split("."))) )
        print([line2,line3,line4][(r%side==0)+(r%base==0)])

In [10]:
answer, board = create_board()
print_board(answer)
print_board(board)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 5 │ 3 │ 2 ║ 7 │ 1 │ 4 ║ 9 │ 6 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 6 │ 8 ║ 5 │ 3 │ 2 ║ 7 │ 1 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 1 │ 4 ║ 9 │ 6 │ 8 ║ 5 │ 3 │ 2 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │ 8 │ 9 ║ 6 │ 2 │ 5 ║ 3 │ 4 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 4 │ 7 ║ 1 │ 8 │ 9 ║ 6 │ 2 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 2 │ 5 ║ 3 │ 4 │ 7 ║ 1 │ 8 │ 9 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 8 │ 5 │ 6 ║ 2 │ 7 │ 3 ║ 4 │ 9 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 9 │ 1 ║ 8 │ 5 │ 6 ║ 2 │ 7 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 7 │ 3 ║ 4 │ 9 │ 1 ║ 8 │ 5 │ 6 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 3 │   ║ 7 │   │ 4 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 9 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 8 │   